Get Raw Batting Data

    - Currently the model averages the recent team hitting performance
    - This does not account for the particular players in the starting lineup that day
    - e.g. If a key hitter is resting, injured, got traded, etc.
    - To begin to model this we first need to scrape the raw batter data (similar to how we got the pitching data)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)

import lxml
import html5lib
from urllib.request import urlopen
import time

from bs4 import BeautifulSoup
import requests

Let's look at Rafael Furcal's url: https://www.retrosheet.org/boxesetc/F/Pfurcr001.htm

In [2]:
def get_daily_season_links_batter(batter_id):
    letter = batter_id.upper()[0]
    url_prefix = 'https://www.retrosheet.org/boxesetc/'
    url = url_prefix+letter+'/P'+batter_id+'.htm'
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)


    body = list(html[2].children)[5]
    pre_texts = [x for x in body.find_all('pre')]
    secnum = np.where([x.get_text().strip().startswith('Batting Record') for x in pre_texts])[0][0]
    a_pre_texts = pre_texts[secnum].find_all('a')
    daily_season_links = [url_prefix+x.attrs['href'][3:] for x in a_pre_texts if x.get_text()=='Daily']
    return(daily_season_links)

In [3]:
get_daily_season_links_batter('furcr001')

['https://www.retrosheet.org/boxesetc/2000/Ifurcr0010012000.htm',
 'https://www.retrosheet.org/boxesetc/2001/Ifurcr0010022001.htm',
 'https://www.retrosheet.org/boxesetc/2002/Ifurcr0010032002.htm',
 'https://www.retrosheet.org/boxesetc/2003/Ifurcr0010042003.htm',
 'https://www.retrosheet.org/boxesetc/2004/Ifurcr0010052004.htm',
 'https://www.retrosheet.org/boxesetc/2005/Ifurcr0010062005.htm',
 'https://www.retrosheet.org/boxesetc/2006/Ifurcr0010072006.htm',
 'https://www.retrosheet.org/boxesetc/2007/Ifurcr0010082007.htm',
 'https://www.retrosheet.org/boxesetc/2008/Ifurcr0010092008.htm',
 'https://www.retrosheet.org/boxesetc/2009/Ifurcr0010102009.htm',
 'https://www.retrosheet.org/boxesetc/2010/Ifurcr0010112010.htm',
 'https://www.retrosheet.org/boxesetc/2011/Ifurcr0010122011.htm',
 'https://www.retrosheet.org/boxesetc/2011/Ifurcr0010132011.htm',
 'https://www.retrosheet.org/boxesetc/2012/Ifurcr0010142012.htm',
 'https://www.retrosheet.org/boxesetc/2014/Ifurcr0010152014.htm']

In [4]:
def get_season_batting_data(url):    
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)[-1]
    body = list(html.children)[-1]
    sec_next = list(body.children)
    secnum = np.where(["Opponent" in str(x) for x in sec_next])[0][0]
    key_section = sec_next[secnum]
    working_part = list(key_section.children)
    p_header = working_part[0].strip().split()
    mod_header= ['at_vs','Opponent','League', 'GS', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'XI', 'ROE', 'GDP',
       'SB', 'CS', 'AVG', 'OBP', 'SLG', 'BP', 'Pos']

    date_list = []
    day_href_list = []
    for k in range(1,len(working_part),4):
        date_list.append(working_part[k].get_text().strip())
        day_href_list.append(working_part[k].attrs['href'])

    dblhead_num_list = []
    for k in range(2,len(working_part),4):
        dblhead_num_list.append(working_part[k].strip())

    game_href_list = []
    for k in range(3,len(working_part),4):
        game_href_list.append(working_part[k].attrs['href'])

    main_data_matrix = []
    for k in range(4,len(working_part),4):
        main_data_row = (working_part[k].strip().split())[:27]
        main_data_matrix.append(main_data_row)
    row_sizes = [len(x) for x in main_data_matrix]
    max_row_size = max(row_sizes)
    min_row_size = min(row_sizes)
    if (min_row_size == max_row_size) and (max_row_size==27):
        # Everything has all 27 columns
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header)
    elif (min_row_size == max_row_size) and (max_row_size==26):
        # Everything has 26 columns, will guess position is missing
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header[:26])
        out_df['Pos'] = ''
    elif (min_row_size == 26) and (max_row_size==27):
        # Guessing position is missing for some rows but not others
        main_data_matrix = [x if len(x)==27 else x+[''] for x in main_data_matrix]
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header)
    else:
        print('finding rows with less than 26 or more than 27 entries - Returning None')
        return(None)
    out_df['date'] = date_list
    out_df['dblhead_num'] = dblhead_num_list
    return(out_df)

In [5]:
# Get all the data for a particular batter

def get_full_batting_data(batter_id):
    link_list = get_daily_season_links_batter(batter_id)
    df_batting = pd.DataFrame()
    for url in link_list:
        df_batting = pd.concat((df_batting, get_season_batting_data(url)))
    return(df_batting)

In [6]:
df_furcal = get_full_batting_data('furcr001')
df_furcal.head()

,at_vs,Opponent,League,GS,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,XI,ROE,GDP,SB,CS,AVG,OBP,SLG,BP,Pos,date,dblhead_num
0,VS,COL,N,1,4,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,.500,.500,.500,8,ss,4- 4-2000,
1,VS,COL,N,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.500,.500,.500,1,"pr,2b",4- 5-2000,
2,VS,SF,N,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.429,.429,.429,9,ss,4- 7-2000,
3,VS,SF,N,1,4,1,1,0,0,0,1,0,0,2,0,1,0,0,0,0,0,0,.364,.364,.364,1,2b,4- 8-2000,
4,VS,SF,N,1,3,3,2,0,1,0,0,1,0,0,0,1,0,0,1,0,2,0,.429,.467,.571,1,ss,4- 9-2000,


In [7]:
df_furcal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1614 entries, 0 to 8
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   at_vs        1614 non-null   object
 1   Opponent     1614 non-null   object
 2   League       1614 non-null   object
 3   GS           1614 non-null   object
 4   AB           1614 non-null   object
 5   R            1614 non-null   object
 6   H            1614 non-null   object
 7   2B           1614 non-null   object
 8   3B           1614 non-null   object
 9   HR           1614 non-null   object
 10  RBI          1614 non-null   object
 11  BB           1614 non-null   object
 12  IBB          1614 non-null   object
 13  SO           1614 non-null   object
 14  HBP          1614 non-null   object
 15  SH           1614 non-null   object
 16  SF           1614 non-null   object
 17  XI           1614 non-null   object
 18  ROE          1614 non-null   object
 19  GDP          1614 non-null   o

In [8]:
df = pd.read_csv('df_bp7.csv', low_memory=False)
df.shape
df.head()

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,Strt_GS_h,Strt_GS_v,Strt_IP_h,Strt_IP_v,Strt_H_h,Strt_H_v,Strt_BFP_h,Strt_BFP_v,Strt_HR_h,Strt_HR_v,Strt_R_h,Strt_R_v,Strt_ER_h,Strt_ER_v,Strt_BB_h,Strt_BB_v,Strt_IB_h,Strt_IB_v,Strt_SO_h,Strt_SO_v,Strt_SH_h,Strt_SH_v,Strt_SF_h,Strt_SF_v,Strt_WP_h,Strt_WP_v,Strt_HBP_h,Strt_HBP_v,Strt_BK_h,Strt_BK_v,Strt_2B_h,Strt_2B_v,Strt_3B_h,Strt_3B_v,Strt_IP_real_h,Strt_IP_real_v,Strt_rollsum_IP_real_3_h,Strt_rollsum_IP_real_3_v,Strt_rollsum_H_3_h,Strt_rollsum_H_3_v,Strt_rollsum_BFP_3_h,Strt_rollsum_BFP_3_v,Strt_rollsum_HR_3_h,Strt_rollsum_HR_3_v,Strt_rollsum_R_3_h,Strt_rollsum_R_3_v,Strt_rollsum_ER_3_h,Strt_rollsum_ER_3_v,Strt_rollsum_BB_3_h,Strt_rollsum_BB_3_v,Strt_rollsum_IB_3_h,Strt_rollsum_IB_3_v,Strt_rollsum_SO_3_h,Strt_rollsum_SO_3_v,Strt_rollsum_SH_3_h,Strt_rollsum_SH_3_v,Strt_rollsum_SF_3_h,Strt_rollsum_SF_3_v,Strt_rollsum_WP_3_h,Strt_rollsum_WP_3_v,Strt_rollsum_HBP_3_h,Strt_rollsum_HBP_3_v,Strt_rollsum_BK_3_h,Strt_rollsum_BK_3_v,Strt_rollsum_2B_3_h,Strt_rollsum_2B_3_v,Strt_rollsum_3B_3_h,Strt_rollsum_3B_3_v,Strt_rollsum_IP_real_14_h,Strt_rollsum_IP_real_14_v,Strt_rollsum_H_14_h,Strt_rollsum_H_14_v,Strt_rollsum_BFP_14_h,Strt_rollsum_BFP_14_v,Strt_rollsum_HR_14_h,Strt_rollsum_HR_14_v,Strt_rollsum_R_14_h,Strt_rollsum_R_14_v,Strt_rollsum_ER_14_h,Strt_rollsum_ER_14_v,Strt_rollsum_BB_14_h,Strt_rollsum_BB_14_v,Strt_rollsum_IB_14_h,Strt_rollsum_IB_14_v,Strt_rollsum_SO_14_h,Strt_rollsum_SO_14_v,Strt_rollsum_SH_14_h,Strt_rollsum_SH_14_v,Strt_rollsum_SF_14_h,Strt_rollsum_SF_14_v,Strt_rollsum_WP_14_h,Strt_rollsum_WP_14_v,Strt_rollsum_HBP_14_h,Strt_rollsum_HBP_14_v,Strt_rollsum_BK_14_h,Strt_rollsum_BK_14_v,Strt_rollsum_2B_14_h,Strt_rollsum_2B_14_v,Strt_rollsum_3B_14_h,Strt_rollsum_3B_14_v,Strt_rollsum_IP_real_30_h,Strt_rollsum_IP_real_30_v,Strt_rollsum_H_30_h,Strt_rollsum_H_30_v,Strt_rollsum_BFP_30_h,Strt_rollsum_BFP_30_v,Strt_rollsum_HR_30_h,Strt_rollsum_HR_30_v,Strt_rollsum_R_30_h,Strt_rollsum_R_30_v,

In [9]:
batter_ids = np.array([])
for num in range(1,10):
    for suffix in ['_h','_v']:
        # Check whether this should be '_id' or '_name'
        colname = 'batter'+str(num)+'_name'+suffix
        batter_ids = np.concatenate((batter_ids, pd.unique(df[colname])))
batter_ids = pd.unique(batter_ids)

In [10]:
len(batter_ids), batter_ids[:10]

(4392,
 array(['hendr001', 'youne001', 'veraq001', 'reesp001', 'castl001',
        'bergp001', 'vinaf001', 'erstd001', 'andeb001', 'walkt002'],
       dtype=object))

In [15]:
# Given to you in /batter_data 
# *** run only if you don't mind waiting ***

# Specify the folder path where you want to save the files
folder_path = '/Volumes/CharmedXi/beatVegas/batter_2000'

for i in range(len(batter_ids)):
    p_id = batter_ids[i]
    print(i, p_id)
    # Construct the file path with the folder path and batter ID
    fname_out = os.path.join(folder_path, 'batting_data_' + p_id + '.csv')
    
    # Check if the file already exists
    if os.path.exists(fname_out):
        print(f'File exists for {p_id}. Skipping...')
        continue
    
    # Fetch batting data
    df_temp = get_full_batting_data(p_id)
    
    # Save the data to CSV
    df_temp.to_csv(fname_out, index=False)

0 hendr001
File exists for hendr001. Skipping...
1 youne001
File exists for youne001. Skipping...
2 veraq001
File exists for veraq001. Skipping...
3 reesp001
File exists for reesp001. Skipping...
4 castl001
File exists for castl001. Skipping...
5 bergp001
File exists for bergp001. Skipping...
6 vinaf001
File exists for vinaf001. Skipping...
7 erstd001
File exists for erstd001. Skipping...
8 andeb001
File exists for andeb001. Skipping...
9 walkt002
File exists for walkt002. Skipping...
10 beckr002
File exists for beckr002. Skipping...
11 clayr001
File exists for clayr001. Skipping...
12 stews002
File exists for stews002. Skipping...
13 womat001
File exists for womat001. Skipping...
14 kendj001
File exists for kendj001. Skipping...
15 chrir001
File exists for chrir001. Skipping...
16 camem001
File exists for camem001. Skipping...
17 hockd001
File exists for hockd001. Skipping...
18 mclem001
File exists for mclem001. Skipping...
19 biggc001
File exists for biggc001. Skipping...
20 nunnj00